In [1]:
import numpy as np
import pandas as pd
#Reading data file:
dataFile = ['userId', 'itemId', 'rating', 'timestamp']
data = pd.read_csv('ml-100k/u.data', sep='\t', names=dataFile,encoding='latin-1')

# data.head()

#Reading items file:
movieInfoFile =['itemId', 'title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movieInfoneedcut = pd.read_csv('ml-100k/u.item', sep='|', names=movieInfoFile,
encoding='latin-1')

movieInfo=pd.DataFrame(movieInfoneedcut.itemId)
movieInfo['title']=movieInfoneedcut['title']
# movieInfo.head()

data=pd.merge(data,movieInfo, left_on= 'itemId', right_on= 'itemId')
data=pd.DataFrame.sort_values(data,['userId','itemId'], ascending=[0,1])
data.head()
# numUsers=max(data.userId)
# numMovies=max(data.itemId)

numUsers=data['userId'].unique().shape[0]
numMovies=data['itemId'].unique().shape[0]

moviePerUser=data.userId.value_counts()
userPerMovie=data.title.value_counts()

def favoriteMovies(activeUser, N):
    topMovies=pd.DataFrame.sort_values(
                data[data.userId==activeUser], ['rating'], ascending=[0])[:N]
    return list(topMovies.title)

print (favoriteMovies(5,3))#3 fav movies of user 5



['Men in Black (1997)', 'Blade Runner (1982)', 'Empire Strikes Back, The (1980)']


In [2]:
userItemRatingMatrix=pd.pivot_table(data, values='rating', index=['userId'], columns=['itemId'])
# userItemRatingMatrix.head()
from scipy.spatial.distance import correlation
def similarity(user1,user2):
#   normalizing
    user1=np.array(user1)-np.nanmean(user1)
    user2=np.array(user2)-np.nanmean(user2)
    commonItemIds=[
        i for i in range(len(user1))
        if user1[i]>0 and user2[i]>0
    ]
    if len(commonItemIds)==0:
        return 0
    else:
        user1-np.array([user1[i] for i in commonItemIds])
        user2-np.array([user2[i] for i in commonItemIds])
        return correlation(user1, user2)
    
    def nearestNeighbourRatings(activeUser, K):
        similarityMatrix=pd.DataFrame(index=userItemRatingMatrix.index, columns=['Similarity'])
        for i in userItemRatingMatrix.indes:
            similarityMatrix.loc[i]=similarity(userItemRatingMatrix.loc[activeUser], userItemRatingMatrix.loc[i])
        similarityMatrix=pd.DataFrame.sort_values(similarityMatrix, ['Similarity'], ascending=[0])
        nearestNeighnours=similarityMatrix[:K]
        neighbourItemRatings=userItemRatingMatrix.loc[nearestNeighbours.index]
        
        predictItemRating=pd.DataFrame(index=userItemRatingMatrix.columns, columns=['Rating'])
        
        for i in userItemRatingMatrix.columns:
            predictedRating=np.nanmean(userItemRatingMatrix.loc[activeUser])
            for j in neighbourItemRatings.index:
                if userItemRatingMatrix.loc[j,i]>0:
                    predictedRating+=(userItemRatingMatrix.loc[j,i]-np.nanmean(userItemRatingMatrix.loc[j]))*nearestNeighbours.loc[j, 'Similarity']
            predictItemRating.loc[i, 'Rating']=predictedRating
        return predictItemRating

In [3]:
movieInfo.columns

Index(['itemId', 'title'], dtype='object')

In [5]:
def matrixFactorization(R, K, steps=10, gamma=0.001, lamda=0.02):
    N=len(R.index)#number of users
    M=len(R.columns)#number of items
    P=pd.DataFrame(np.random.rand(N,K), index=R.index)
    Q=pd.DataFrame(np.random.rand(M,K), index=R.columns)
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    e=e+pow(R.loc[i,j]-np.dot(P.loc[i], Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        print (step)
    return P,Q

(P,Q)=matrixFactorization(userItemRatingMatrix.iloc[:100,:100], K=2, gamma=0.001, lamda=0.02, steps=10)

activeUser=1
predictItemRating=pd.DataFrame(np.dot(P.loc[activeUser],Q.T), index=Q.index, columns=['Ratings'])
topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Ratings'],ascending=[0])[:3]


0
1
2
3
4
5
6
7
8
9


In [6]:
#we found the ratings of all movie by the active user and then sorted them to top 3
topRecommendationsTitles=movieInfo.loc[movieInfo.itemId.isin(topRecommendations.index)]
print (list(topRecommendationsTitles.title))

['Twelve Monkeys (1995)', 'Star Wars (1977)', 'Fargo (1996)']


In [7]:
topRecommendations

,Ratings
itemId,
50,5.636208
7,5.313892
100,4.861698


In [1]:
import pandas as pd
df1 = pd.DataFrame({'itemId': [50, 7, 100, 5, 2], 'Ratings': [3.3, 2.7, 2.5, 2.43, 2.33]})
df2 = pd.DataFrame({'itemId': [50, 7, 100, 3, 80], 'Label': [3.3, 2.7, 2.5, 2.49, 2.33]})

In [3]:
pointlist = [5,4,3,2,1]
df1['points1'] = pointlist
df2['points2'] = pointlist

In [4]:
df1 = pd.merge(
    df1, df2, how='outer',
    left_on='itemId', right_on='itemId')
df1

,Ratings,itemId,points_x,points1,Label,points_y,points2
0,3.30,50,5.0,5.0,3.30,5.0,5.0
1,2.70,7,4.0,4.0,2.70,4.0,4.0
2,2.50,100,3.0,3.0,2.50,3.0,3.0
3,2.43,5,2.0,2.0,NaN,NaN,NaN
4,2.33,2,1.0,1.0,NaN,NaN,NaN
5,NaN,3,NaN,NaN,2.49,2.0,2.0
6,NaN,80,NaN,NaN,2.33,1.0,1.0


In [6]:

df1 = df1[['itemId', 'Ratings', 'points1', 'points2' ]]
df1 = df1.fillna(0)

In [8]:
df1

,itemId,Ratings,points1,points2
0,50,3.30,5.0,5.0
1,7,2.70,4.0,4.0
2,100,2.50,3.0,3.0
3,5,2.43,2.0,0.0
4,2,2.33,1.0,0.0
5,3,0.00,0.0,2.0
6,80,0.00,0.0,1.0


In [11]:

df1['result'] = df1.apply(
    lambda row: (row['points1']+row['points1'])/2,
    axis=1
)

In [12]:
df1.sort_values(by=['result'],ascending=[0])

,itemId,Ratings,points1,points2,result
0,50,3.30,5.0,5.0,5.0
1,7,2.70,4.0,4.0,4.0
2,100,2.50,3.0,3.0,3.0
3,5,2.43,2.0,0.0,2.0
4,2,2.33,1.0,0.0,1.0
5,3,0.00,0.0,2.0,0.0
6,80,0.00,0.0,1.0,0.0
